In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

import statsmodels.api as sm

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# import bcb

# https://dadosabertos.bcb.gov.br/dataset/20542-saldo-da-carteira-de-credito-com-recursos-livres---total/resource/6e2b0c97-afab-4790-b8aa-b9542923cf88
# https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries
# https://api.bcb.gov.br/dados/serie/bcdata.sgs.20542/dados?formato=json&dataInicial=01/01/2010&dataFinal=31/12/2016

In [2]:
acoes = ['ITUB4.SA', 'BBAS3.SA', 'MGLU3.SA', 'BHIA3.SA', 'PETR4.SA', 'PRIO3.SA', 'VALE3.SA', 'CMIN3.SA', 'B3SA3.SA']

start_date = '2022-01-01'
end_date = '2024-05-31'

data_inicial = '01/01/2022'
data_final = '31/05/2024'

In [3]:
dados_acoes = yf.download(acoes, start_date, end_date, interval='1mo')

df_acoes = pd.DataFrame(dados_acoes)
df_acoes = df_acoes[['Adj Close']].droplevel(level=0, axis=1)

[*********************100%%**********************]  9 of 9 completed


In [4]:
df_retornos = df_acoes.pct_change()
df_retornos.columns = [f'Retorno {coluna}' for coluna in df_retornos.columns]

In [5]:
def extrair_dados(serie, data_inicio, data_fim):
    url = 'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json&dataInicial={}&dataFinal={}'.format(serie, data_inicio, data_fim)

    df = pd.read_json(url)
    df.columns = ['Data', 'Valor']
    df.set_index('Data', inplace=True)
    df.index = pd.to_datetime(df.index, dayfirst=True)

    return df

In [6]:
# inadimplência mensal 21084
df_inad = extrair_dados(21084, data_inicial, data_final)
df_inad = df_inad.rename(columns={'Valor': 'Inad.'})
df_inad['Var. Inad.'] = df_inad['Inad.'].pct_change()

# selic acumulada mensal 4390
df_selic = extrair_dados(4390, data_inicial, data_final)
df_selic = df_selic.rename(columns={'Valor': 'Selic'})
df_selic['Var. Selic'] = df_selic['Selic'].pct_change()

# meta selic diária 432
# df_meta_selic = df_meta_selic.rename(columns={'Valor': 'Meta Selic'})
# df_meta_selic = extrair_dados(432, data_inicial, data_final)

# ipca mensal 433
df_ipca = extrair_dados(433, data_inicial, data_final)
df_ipca = df_ipca.rename(columns={'Valor': 'IPCA'})
df_ipca['Var. IPCA'] = df_ipca['IPCA'].pct_change()

# igpm mensal 189
df_igpm = extrair_dados(189, data_inicial, data_final)
df_igpm = df_igpm.rename(columns={'Valor': 'IGPM'})
df_igpm['Var. IGPM'] = df_igpm['IGPM'].pct_change()

# inpc mensal 188
df_inpc = extrair_dados(188, data_inicial, data_final)
df_inpc = df_inpc.rename(columns={'Valor': 'INPC'})
df_inpc['Var. INPC'] = df_inpc['INPC'].pct_change()

In [7]:
# https://fred.stlouisfed.org/series/PIORECRUSDM

df_ferro = pd.read_excel('FRED - Iron Ore.xls').dropna()

df_ferro = df_ferro.drop(df_ferro.index[0:2])
df_ferro.columns = ['Data', 'Ferro']
df_ferro.set_index('Data', inplace=True)
df_ferro.index = pd.to_datetime(df_ferro.index)

df_ferro['Var. Ferro'] = df_ferro['Ferro'].pct_change()

c:\Users\Felip\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
C:\Users\Felip\AppData\Local\Temp\ipykernel_5480\1814205123.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_ferro['Var. Ferro'] = df_ferro['Ferro'].pct_change()


In [9]:
# https://www.opec.org/opec_web/en/data_graphs/40.htm
# https://www.statista.com/statistics/277914/monthly-average-prices-of-the-opec-crude-oil-basket/

df_petroleo = pd.read_excel("OPEP - Petroleo.xlsx", skiprows=3)

df_petroleo.set_index('Data', inplace=True)
df_petroleo.index = pd.to_datetime(df_petroleo.index)
df_petroleo['Var. Petroleo'] = df_petroleo['Petroleo'].pct_change()

In [10]:
df_cap_est = pd.read_excel('Capital Estrangeiro (em MM).xlsx')

df_cap_est = df_cap_est.rename(columns={'Mês / Month': 'Data', 'Compras / Buyers (A)': 'Compras', 'Vendas / Sellers (B)': 'Vendas', 'IPO / Follow On (C)': 'IPO', 'A+C-B (Saldo) / A+C-B (Balance)': 'Saldo'}).set_index('Data').dropna()
df_cap_est = df_cap_est.drop(df_cap_est.index[-1])
df_cap_est.index = pd.to_datetime(df_cap_est.index)
df_cap_est = df_cap_est[['Saldo']]

df_cap_est['Var. Saldo'] = df_cap_est['Saldo'].pct_change()

In [11]:
df_acoes = df_acoes.drop(df_acoes.index[-1])
df_retornos = df_retornos.drop(df_retornos.index[-1])
df_selic = df_selic.drop(df_selic.index[-1])
df_ipca = df_ipca.drop(df_ipca.index[-1])
df_igpm = df_igpm.drop(df_igpm.index[-1])
df_inpc = df_inpc.drop(df_inpc.index[-1])

In [12]:
print(df_acoes.shape)
print(df_retornos.shape)
print(df_inad.shape)
print(df_selic.shape)
print(df_ipca.shape)
print(df_igpm.shape)
print(df_inpc.shape)
print(df_ferro.shape)
print(df_petroleo.shape)
print(df_cap_est.shape)

(28, 9)
(28, 9)
(28, 2)
(28, 2)
(28, 2)
(28, 2)
(28, 2)
(28, 2)
(28, 2)
(16, 2)


In [13]:
df_bancario_inad = pd.concat([df_acoes, df_retornos, df_inad], axis=1)[['ITUB4.SA', 'Retorno ITUB4.SA', 'BBAS3.SA', 'Retorno BBAS3.SA', 'Inad.', 'Var. Inad.']]
df_varejo_selic_inflacao = pd.concat([df_acoes, df_retornos, df_selic, df_ipca, df_igpm, df_inpc], axis=1)[['MGLU3.SA', 'Retorno MGLU3.SA', 'BHIA3.SA', 'Retorno BHIA3.SA', 'Selic', 'Var. Selic', 'IPCA', 'Var. IPCA', 'IGPM', 'Var. IGPM', 'INPC', 'Var. INPC']]
df_petrolifero_petroleo = pd.concat([df_acoes, df_retornos, df_petroleo], axis=1)[['PETR4.SA', 'Retorno PETR4.SA', 'PRIO3.SA', 'Retorno PRIO3.SA', 'Petroleo', 'Var. Petroleo']]
df_minerador_ferro = pd.concat([df_acoes, df_retornos, df_ferro], axis=1)[['VALE3.SA', 'Retorno VALE3.SA', 'CMIN3.SA', 'Retorno CMIN3.SA', 'Ferro', 'Var. Ferro']]
df_institucional_cap_est = pd.concat([df_acoes, df_retornos, df_cap_est], axis=1)[['B3SA3.SA', 'Retorno B3SA3.SA', 'Saldo', 'Var. Saldo']].dropna()

In [14]:
# df_bancario_inad.to_excel('Bancario_Inad.xlsx')
# df_varejo_selic_inflacao.to_excel('Varejo_Selic_Inflacao.xlsx')
# df_petrolifero_petroleo.to_excel('Petrolifero_Petroleo.xlsx')
# df_minerador_ferro.to_excel('Minerador_Ferro.xlsx')
# df_institucional_cap_est.to_excel('Institucional_CapEst.xlsx')

In [15]:
df_bancario_inad.corr()

,ITUB4.SA,Retorno ITUB4.SA,BBAS3.SA,Retorno BBAS3.SA,Inad.,Var. Inad.
ITUB4.SA,1.000000,0.294376,0.901722,-0.073165,0.025184,-0.458137
Retorno ITUB4.SA,0.294376,1.000000,0.211363,0.531623,0.073891,0.030201
BBAS3.SA,0.901722,0.211363,1.000000,0.087767,0.261894,-0.559167
Retorno BBAS3.SA,-0.073165,0.531623,0.087767,1.000000,0.169584,0.357205
Inad.,0.025184,0.073891,0.261894,0.169584,1.000000,-0.161130
Var. Inad.,-0.458137,0.030201,-0.559167,0.357205,-0.161130,1.000000


In [16]:
df_varejo_selic_inflacao.corr()

,MGLU3.SA,Retorno MGLU3.SA,BHIA3.SA,Retorno BHIA3.SA,Selic,Var. Selic,IPCA,Var. IPCA,IGPM,Var. IGPM,INPC,Var. INPC
MGLU3.SA,1.000000,0.245041,0.923384,0.215515,-0.179391,0.203730,0.386592,-0.119095,0.328449,0.083087,0.434848,-0.091204
Retorno MGLU3.SA,0.245041,1.000000,0.161955,0.669625,0.245162,0.056650,-0.226999,-0.120920,-0.162434,-0.095266,-0.212199,-0.003248
BHIA3.SA,0.923384,0.161955,1.000000,0.314356,0.026091,0.285406,0.186003,-0.279223,0.245396,0.127361,0.237328,-0.196186
Retorno BHIA3.SA,0.215515,0.669625,0.314356,1.000000,0.243572,0.283175,-0.228667,-0.219597,-0.245147,-0.130493,-0.181521,0.009726
Selic,-0.179391,0.245162,0.026091,0.243572,1.000000,0.449621,-0.437537,-0.325896,-0.524849,0.161864,-0.447308,-0.364759
Var. Selic,0.203730,0.056650,0.285406,0.283175,0.449621,1.000000,0.037943,-0.013241,0.131261,0.412173,0.124489,-0.044178
IPCA,0.386592,-0.226999,0.186003,-0.228667,-0.437537,0.037943,1.000000,0.437769,0.539561,-0.082660,0.981223,0.354761
Var. IPCA,-0.119095,-0.120920,-0.279223,-0.219597,-0.325896,-0.013241,0.437769,1.000000,0.477263,-0.124856,0.462207,0.525725
IGPM,0.328449,-0.162434,0.245396,-0.245147,-0.524849,0.131261,0.539561,0.477263,1.000000,0.217353,0.552448,0.353725
Var. IGPM,0.083087,-0.095266,0.127361,-0.130493,0.161864,0.412173,-0.082660,-0.124856,0.217353,1.000000,-0.060831,-0.008573


In [17]:
df_petrolifero_petroleo.corr()

,PETR4.SA,Retorno PETR4.SA,PRIO3.SA,Retorno PRIO3.SA,Petroleo,Var. Petroleo
PETR4.SA,1.000000,0.197919,0.856438,-0.041163,-0.431009,0.013970
Retorno PETR4.SA,0.197919,1.000000,0.055038,0.066059,-0.056961,-0.014247
PRIO3.SA,0.856438,0.055038,1.000000,0.134714,-0.599709,0.018273
Retorno PRIO3.SA,-0.041163,0.066059,0.134714,1.000000,-0.083599,-0.036848
Petroleo,-0.431009,-0.056961,-0.599709,-0.083599,1.000000,0.285587
Var. Petroleo,0.013970,-0.014247,0.018273,-0.036848,0.285587,1.000000


In [18]:
df_minerador_ferro.corr()

,VALE3.SA,Retorno VALE3.SA,CMIN3.SA,Retorno CMIN3.SA,Ferro,Var. Ferro
VALE3.SA,1.000000,0.424968,0.127954,0.376571,0.450074,0.541828
Retorno VALE3.SA,0.424968,1.000000,0.015297,0.468631,-0.150836,0.356818
CMIN3.SA,0.127954,0.015297,1.000000,0.253633,0.489558,0.263597
Retorno CMIN3.SA,0.376571,0.468631,0.253633,1.000000,-0.115404,0.622778
Ferro,0.450074,-0.150836,0.489558,-0.115404,1.000000,0.306078
Var. Ferro,0.541828,0.356818,0.263597,0.622778,0.306078,1.000000


In [19]:
df_institucional_cap_est.corr()

,B3SA3.SA,Retorno B3SA3.SA,Saldo,Var. Saldo
B3SA3.SA,1.000000,0.504563,0.518258,-0.113903
Retorno B3SA3.SA,0.504563,1.000000,0.716351,-0.653798
Saldo,0.518258,0.716351,1.000000,-0.480137
Var. Saldo,-0.113903,-0.653798,-0.480137,1.000000


In [20]:
modelo = sm.OLS(df_varejo_selic_inflacao['MGLU3.SA'], df_varejo_selic_inflacao[['IPCA', 'IGPM', 'INPC']])
resultado = modelo.fit()

print(resultado.summary())

                                 OLS Regression Results                                
Dep. Variable:               MGLU3.SA   R-squared (uncentered):                   0.576
Model:                            OLS   Adj. R-squared (uncentered):              0.525
Method:                 Least Squares   F-statistic:                              11.33
Date:                Fri, 21 Jun 2024   Prob (F-statistic):                    6.99e-05
Time:                        21:02:55   Log-Likelihood:                         -127.13
No. Observations:                  28   AIC:                                      260.3
Df Residuals:                      25   BIC:                                      264.3
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [21]:
modelo = sm.OLS(df_varejo_selic_inflacao['BHIA3.SA'], df_varejo_selic_inflacao[['IPCA', 'IGPM', 'INPC']])
resultado = modelo.fit()

print(resultado.summary())

                                 OLS Regression Results                                
Dep. Variable:               BHIA3.SA   R-squared (uncentered):                   0.453
Model:                            OLS   Adj. R-squared (uncentered):              0.387
Method:                 Least Squares   F-statistic:                              6.900
Date:                Fri, 21 Jun 2024   Prob (F-statistic):                     0.00153
Time:                        21:02:55   Log-Likelihood:                         -141.89
No. Observations:                  28   AIC:                                      289.8
Df Residuals:                      25   BIC:                                      293.8
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [22]:
modelo = sm.OLS(df_institucional_cap_est['Retorno B3SA3.SA'], df_institucional_cap_est[['Saldo']])
resultado = modelo.fit()

print(resultado.summary())

                                 OLS Regression Results                                
Dep. Variable:       Retorno B3SA3.SA   R-squared (uncentered):                   0.507
Model:                            OLS   Adj. R-squared (uncentered):              0.472
Method:                 Least Squares   F-statistic:                              14.42
Date:                Fri, 21 Jun 2024   Prob (F-statistic):                     0.00196
Time:                        21:02:55   Log-Likelihood:                          17.173
No. Observations:                  15   AIC:                                     -32.35
Df Residuals:                      14   BIC:                                     -31.64
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

c:\Users\Felip\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_stats_py.py:1971: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  k, _ = kurtosistest(a, axis)


In [35]:
# Preços de Itaú e BB e inadimplência absolutos

fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=df_bancario_inad.index, y=df_bancario_inad['ITUB4.SA'], name='ITUB4'))
fig.add_trace(go.Scatter(x=df_bancario_inad.index, y=df_bancario_inad['BBAS3.SA'], name='BBAS3'))

fig.add_trace(go.Scatter(x=df_bancario_inad.index, y=df_bancario_inad['Inad.'], name='Inadimplência'), secondary_y=True)

fig.update_layout(xaxis_title="Data", yaxis_title="Preço da Ação (R$)", yaxis2_title="Inadimplência da Carteira de Crédito PF (%)", template='plotly_dark', title="Análise Gráfica | Setor Bancário x Inadimplência", width=1000, height=600)

fig.show()

In [36]:
# Preços de Itaú e BB e inadimplência em variação percentual

fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=df_bancario_inad.index, y=df_bancario_inad['Retorno ITUB4.SA'], name='ITUB4'))
fig.add_trace(go.Scatter(x=df_bancario_inad.index, y=df_bancario_inad['Retorno BBAS3.SA'], name='BBAS3'))

fig.add_trace(go.Scatter(x=df_bancario_inad.index, y=df_bancario_inad['Var. Inad.'], name='Inadimplência'))

fig.update_layout(xaxis_title="Data", yaxis_title="Variação Percentual", yaxis2_title="Variação da Inadimplência da Carteira de Crédito PF (%)", template='plotly_dark', title="Análise Gráfica | Setor Bancário x Inadimplência", width=1000, height=600)

fig.show()

In [37]:
# Preços de Magazine Luiza e Casas Bahia e Selic absolutos

fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=df_varejo_selic_inflacao.index, y=df_varejo_selic_inflacao['MGLU3.SA'], name='MGLU3'))
fig.add_trace(go.Scatter(x=df_varejo_selic_inflacao.index, y=df_varejo_selic_inflacao['BHIA3.SA'], name='BHIA3'))

fig.add_trace(go.Scatter(x=df_varejo_selic_inflacao.index, y=df_varejo_selic_inflacao['Selic'], name='Selic'), secondary_y=True)

fig.update_layout(xaxis_title="Data", yaxis_title="Preço da Ação (R$)", yaxis2_title="Selic Acumulada Mensal (%)", template='plotly_dark', title="Análise Gráfica | Setor Varejista x Taxa Selic", width=1000, height=600)

fig.show()

In [38]:
# Preços de Itaú e BB e inadimplência em variação percentual

fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=df_varejo_selic_inflacao.index, y=df_varejo_selic_inflacao['Retorno MGLU3.SA'], name='MGLU3'))
fig.add_trace(go.Scatter(x=df_varejo_selic_inflacao.index, y=df_varejo_selic_inflacao['Retorno BHIA3.SA'], name='BHIA3'))

fig.add_trace(go.Scatter(x=df_varejo_selic_inflacao.index, y=df_varejo_selic_inflacao['Var. Selic'], name='Selic'))

fig.update_layout(xaxis_title="Data", yaxis_title="Variação Percentual", yaxis2_title="Variação da Selic Acumulada Mensal (%)", template='plotly_dark', title="Análise Gráfica | Setor Varejista x Taxa Selic", width=1000, height=600)

fig.show()

In [39]:
# Preços de Magazine Luiza e Casas Bahia e inflação absolutos

fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=df_varejo_selic_inflacao.index, y=df_varejo_selic_inflacao['MGLU3.SA'], name='MGLU3'))
fig.add_trace(go.Scatter(x=df_varejo_selic_inflacao.index, y=df_varejo_selic_inflacao['BHIA3.SA'], name='BHIA3'))

fig.add_trace(go.Scatter(x=df_varejo_selic_inflacao.index, y=df_varejo_selic_inflacao['IPCA'], name='IPCA'), secondary_y=True)
fig.add_trace(go.Scatter(x=df_varejo_selic_inflacao.index, y=df_varejo_selic_inflacao['IGPM'], name='IGPM'), secondary_y=True)
fig.add_trace(go.Scatter(x=df_varejo_selic_inflacao.index, y=df_varejo_selic_inflacao['INPC'], name='INPC'), secondary_y=True)

fig.update_layout(xaxis_title="Data", yaxis_title="Preço da Ação (R$)", yaxis2_title="Inflação Mensal (%)", template='plotly_dark', title="Análise Gráfica | Setor Varejista x Inflação", width=1000, height=600)

fig.show()

In [40]:
# Preços de Magazine Luiza e Casas Bahia e inflação em variação percentual

fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=df_varejo_selic_inflacao.index, y=df_varejo_selic_inflacao['Retorno MGLU3.SA'], name='MGLU3'))
fig.add_trace(go.Scatter(x=df_varejo_selic_inflacao.index, y=df_varejo_selic_inflacao['Retorno BHIA3.SA'], name='BHIA3'))

fig.add_trace(go.Scatter(x=df_varejo_selic_inflacao.index, y=df_varejo_selic_inflacao['Var. IPCA'], name='IPCA'))
fig.add_trace(go.Scatter(x=df_varejo_selic_inflacao.index, y=df_varejo_selic_inflacao['Var. IGPM'], name='IGPM'))
fig.add_trace(go.Scatter(x=df_varejo_selic_inflacao.index, y=df_varejo_selic_inflacao['Var. INPC'], name='INPC'))

fig.update_layout(xaxis_title="Data", yaxis_title="Variação Percentual", yaxis2_title="Variação da Inflação Mensal (%)", template='plotly_dark', title="Análise Gráfica | Setor Varejista x Inflação", width=1000, height=600)

fig.show()

In [41]:
# Preços de Petrobras e PetroRio e petróleo absolutos

fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=df_petrolifero_petroleo.index, y=df_petrolifero_petroleo['PETR4.SA'], name='PETR4'))
fig.add_trace(go.Scatter(x=df_petrolifero_petroleo.index, y=df_petrolifero_petroleo['PRIO3.SA'], name='PRIO3'))

fig.add_trace(go.Scatter(x=df_petrolifero_petroleo.index, y=df_petrolifero_petroleo['Petroleo'], name='Petróleo'), secondary_y=True)

fig.update_layout(xaxis_title="Data", yaxis_title="Preço da Ação (R$)", yaxis2_title="Preço do Barril de Petróleo (US$)", template='plotly_dark', title="Análise Gráfica | Setor Petrolífero x Petróleo", width=1000, height=600)

fig.show()

In [42]:
# Preços de Petrobras e PetroRio e petróleo em variação percentual

fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=df_petrolifero_petroleo.index, y=df_petrolifero_petroleo['Retorno PETR4.SA'], name='PETR4'))
fig.add_trace(go.Scatter(x=df_petrolifero_petroleo.index, y=df_petrolifero_petroleo['Retorno PRIO3.SA'], name='PRIO3'))

fig.add_trace(go.Scatter(x=df_petrolifero_petroleo.index, y=df_petrolifero_petroleo['Var. Petroleo'], name='Var. Petroleo'))

fig.update_layout(xaxis_title="Data", yaxis_title="Variação Percentual", yaxis2_title="Variação do Preço do Barril de Petróleo (%)", template='plotly_dark', title="Análise Gráfica | Setor Petrolífero x Petróleo", width=1000, height=600)

fig.show()

In [43]:
# Preços de Vale e CSN Mineração e ferro absolutos

fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=df_minerador_ferro.index, y=df_minerador_ferro['VALE3.SA'], name='VALE3'))
fig.add_trace(go.Scatter(x=df_minerador_ferro.index, y=df_minerador_ferro['CMIN3.SA'], name='CMIN3'))

fig.add_trace(go.Scatter(x=df_minerador_ferro.index, y=df_minerador_ferro['Ferro'], name='Ferro'), secondary_y=True)

fig.update_layout(xaxis_title="Data", yaxis_title="Preço da Ação (R$)", yaxis2_title="Preço do Ferro (US$)", template='plotly_dark', title="Análise Gráfica | Setor Minerador x Ferro", width=1000, height=600)

fig.show()

In [44]:
# Preços de Vale e CSN Mineração e ferro em variação percentual

fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=df_minerador_ferro.index, y=df_minerador_ferro['Retorno VALE3.SA'], name='VALE3'))
fig.add_trace(go.Scatter(x=df_minerador_ferro.index, y=df_minerador_ferro['Retorno CMIN3.SA'], name='CMIN3'))

fig.add_trace(go.Scatter(x=df_minerador_ferro.index, y=df_minerador_ferro['Var. Ferro'], name='Var. Ferro'))

fig.update_layout(xaxis_title="Data", yaxis_title="Variação Percentual", yaxis2_title="Variação do Preço do Ferro (%)", template='plotly_dark', title="Análise Gráfica | Setor Minerador x Ferro", width=1000, height=600)

fig.show()

In [45]:
# Preços de B3 e capital estrangeiro absolutos

fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=df_institucional_cap_est.index, y=df_institucional_cap_est['B3SA3.SA'], name='B3SA3'))

fig.add_trace(go.Scatter(x=df_institucional_cap_est.index, y=df_institucional_cap_est['Saldo'], name='Capital Estrangeiro'), secondary_y=True)

fig.update_layout(xaxis_title="Data", yaxis_title="Preço da Ação (R$)", yaxis2_title="Saldo do Capital Estrangeiro (R$)", template='plotly_dark', title="Análise Gráfica | B3 x Capital Estrangeiro", width=1000, height=600)

fig.show()

In [46]:
# Preços de B3 e capital estrangeiro em variação percentual

fig = make_subplots(specs=[[{'secondary_y': True}]])

fig.add_trace(go.Scatter(x=df_institucional_cap_est.index, y=df_institucional_cap_est['Retorno B3SA3.SA'], name='B3SA3'))

fig.add_trace(go.Scatter(x=df_institucional_cap_est.index, y=df_institucional_cap_est['Var. Saldo'], name='Var. Capital Estrangeiro'), secondary_y=True)

fig.update_layout(xaxis_title="Data", yaxis_title="Retorno da Ação da B3", yaxis2_title="Variação do Saldo do Capital Estrangeiro (%)", template='plotly_dark', title="Análise Gráfica | B3 x Capital Estrangeiro", width=1000, height=600)

fig.show()